<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->




# **P1 - RA ÍMPAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Erick Eiji Nagao, 21.00690-3' #@param {type:"string"}






In [193]:
import pandas as pd
import numpy as np

In [194]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [195]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [197]:
host_counts = df.groupby(['host_id', 'host_name']).size().reset_index(name='imoveis')

host_counts = host_counts.sort_values(by='imoveis', ascending=False)

top_hosts = host_counts.head(3)

print(top_hosts)

       host_id     host_name  imoveis
713  219517861  Sonder (NYC)        7
394   30283594          Kara        4
605  120762452       Stanley        4


# Q2B. Preço nos bairros com mais imóveis

 Qual o preço médio dos imóveis, considerando somente os preços dos imóveis cujos bairros (`neighbourhood`) tenham mais de 20 imóveis sendo anunciados?  


In [198]:
...
bairros_com_mais_de_20 = df['neighbourhood'].value_counts()
bairros_com_mais_de_20 = bairros_com_mais_de_20[bairros_com_mais_de_20 > 20].index
preco_medio = df[df['neighbourhood'].isin(bairros_com_mais_de_20)]['price'].mean()

print(f'O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD {preco_medio:}')

O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD 151.41888619854723


# Q3B.  Brooklyn *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [199]:
...
q1 = df['price'].quantile(0.25)
q3 = df['price'].quantile(0.75)
iqr = q3 - q1
limite_superior = q3 + 1.5 * iqr

outliers_brooklyn = df[(df['neighbourhood_group'] == 'Brooklyn') & (df['price'] > limite_superior)]
perc_Brooklyn = (len(outliers_brooklyn) / len(df[df['neighbourhood_group'] == 'Brooklyn'])) * 100

print( f'Percentuais de outliers de preços, Brooklyn: {perc_Brooklyn:} %')


Percentuais de outliers de preços, Brooklyn: 3.939393939393939 %


# Q4B. Scores

Nesta tarefa você deve atribuir a cada imóvel um valor de *score* (`df['score']`) baseado nas seguintes regras:

1. Imóveis com `availability_365` > 90 recebem +10 pontos enquanto imóveis com `availability_365` < 5 recebem -5 pontos;
2. Imóveis com o número de reviews >= 10 recebem +5 pontos para cada 10 reviews (por ex. se número de reviews = 20, são atribuídos +10 pontos). Se número de reviews = 0 o imóvel recebe -5 pontos;
3. Imóveis com preço abaixo da média de seu `neighbourhood`	recebem +5 pontos.

Implemente empregando `apply()` ou um laço de programa, como preferir.

In [200]:
...
def calcular_score(row):
    score = 0
    if row['availability_365'] > 90:
        score += 10
    elif row['availability_365'] < 5:
        score -= 5

    if row['number_of_reviews'] >= 10:
        score += (row['number_of_reviews'] // 10) * 5
    elif row['number_of_reviews'] == 0:
        score -= 5

    media_preco_bairro = df[df['neighbourhood'] == row['neighbourhood']]['price'].mean()
    if row['price'] < media_preco_bairro:
        score += 5

    return score

df['score'] = df.apply(calcular_score, axis=1)

print(f'A soma dos valores de score é: ', df['score'].sum())


A soma dos valores de score é:  10955


# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [201]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [202]:
df_taxes['value'] = df_taxes['value'].fillna(0)
df_taxes_pivot = df_taxes.pivot(index='id', columns='type', values='value').reset_index()

df = df.merge(df_taxes_pivot, on='id', how='left')
df[['taxes', 'fines']] = df[['taxes', 'fines']].fillna(0)

media_taxes_fines = df.groupby('neighbourhood_group')[['taxes', 'fines']].mean()

print(media_taxes_fines)

                          taxes       fines
neighbourhood_group                        
Bronx                808.500000  120.150000
Brooklyn             798.457576  292.554545
Manhattan            855.029891  302.997283
Queens               793.987342  318.683544
Staten Island        992.142857  498.285714


# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [204]:
df['line_id'] = range(1, len(df) + 1)

df = df.sort_values(by=['neighbourhood_group', 'neighbourhood'])

df['page_id'] = (df['line_id'] - 1) // 30 + 1

df_report = df.groupby(['page_id', 'neighbourhood_group', 'neighbourhood']).agg({'price': 'mean'}).reset_index()

group_means = df.groupby('neighbourhood_group')['price'].mean().reset_index()
group_means['neighbourhood'] = 'MEAN'
group_means['page_id'] = None
group_means['line_id'] = None
df_report = pd.concat([df_report, group_means], ignore_index=True)

df_report = df_report[['neighbourhood_group', 'neighbourhood', 'price', 'page_id', 'line_id']]

sum_price_pages = df_report[df_report['page_id'].isin([1, 24, 25, 26, 28])]['price'].sum()

print(f'A soma dos preços das páginas 1, 24, 25, 26 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25, 26 e 28 é: 5223.938502673797


#### Aqui um *sample* do report.

In [205]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price page_id line_id
130       Staten Island  Randall Manor   99.000000      27     NaN
131       Staten Island    South Beach   60.000000      27     NaN
132       Staten Island     St. George  130.000000      27     NaN
133       Staten Island      Todt Hill  135.000000      27     NaN
134       Staten Island  Tompkinsville   59.000000      27     NaN
135               Bronx           MEAN   71.500000    None    None
136            Brooklyn           MEAN  128.087879    None    None
137           Manhattan           MEAN  196.157609    None    None
138              Queens           MEAN   92.227848    None    None
139       Staten Island           MEAN   89.428571    None    None

7) Tratar valores ausentes, preenchendo ou removendo dados inconsistentes /  Corrigir erros e inconsistências nos dados / Combinar dados de diferentes fontes em um único conjunto / Criar novas variáveis úteis a partir dos dados existentes / Selecionar apenas os atributos relevantes para a análise / Ajustar o formato dos dados para facilitar o uso nos modelos.